In [ ]:
!pip install langchain langchain_community langchain_core langchain_openai langchain_mongodb pymongo pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters

In [ ]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers.openai_functions import (
    create_metadata_tagger,
)
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata

In [ ]:
os.environ["MONGODB_URI"] = userdata.get("MONGODB_URI")
mongo_uri = os.getenv("MONGODB_URI")

client = MongoClient(mongo_uri)
dbName = "rag"
collectionName = "atlas"

collection = client[dbName][collectionName]

In [ ]:
loader = PyPDFLoader("document.pdf")
pages = loader.load()
cleaned_pages = []

In [ ]:
for page in pages:
    if len(page.page_content.split(" ")) > 20:
        cleaned_pages.append(page)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)

In [ ]:
schema = {
    "properties": {
        "title": {"type": "string"},
        "keywords": {"type": "array", "items": {"type": "string"}},
        "hasCode": {"type": "boolean"},
    },
    "required": ["title", "keywords", "hasCode"],
}

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    temperature=0,
    model="gpt-4o-mini",
)

In [ ]:
document_transformer = create_metadata_tagger(metadata_schema=schema, llm=llm)

In [ ]:
docs = document_transformer.transform_documents(cleaned_pages)

In [ ]:
split_docs = text_splitter.split_documents(docs)

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_type=openai_api_key, model="text-embedding-3-small")

In [ ]:
vectorStore = MongoDBAtlasVectorSearch.from_documents(
    split_docs, embeddings, collection=collection
)

In [ ]:
index = "vector_index"

In [ ]:
vectorStore = MongoDBAtlasVectorSearch.from_connection_string(
    mongo_uri,
    dbName + "." + collectionName,
    OpenAIEmbeddings(disallowed_special=(), openai_api_key=openai_api_key, model="text-embedding-3-small"),
    index_name=index,
)

In [ ]:
def query_data(query):
    retriever = vectorStore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "pre_filter": { "hasCode": { "$eq": False } },
            "score_threshold": 0.01
        },
    )

    template = """
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Do not answer the question if there is no given context.
    Do not answer the question if it is not related to the context.
    Do not give recommendations to anything other than MongoDB.
    Context:
    {context}
    Question: {question}
    """

    custom_rag_prompt = PromptTemplate.from_template(template)

    retrieve = {
        "context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
        "question": RunnablePassthrough()
        }

    llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0, model="gpt-4o-mini")

    response_parser = StrOutputParser()

    rag_chain = (
        retrieve
        | custom_rag_prompt
        | llm
        | response_parser
    )

    answer = rag_chain.invoke(query)


    return answer

In [48]:
print("\n".join(query_data("What is PrintHub?").split(". ")))

PrintHub is an innovative online platform designed to connect users with printing presses, allowing them to find, compare, and order printing services conveniently
It streamlines the discovery process by providing a centralized location where users can compare product offerings, minimum quantities, and costs efficiently
The platform aims to save time, reduce frustration, and modernize the traditional printing process by empowering both sellers and buyers with a user-friendly solution for all printing needs.
